## Data Cleaning Goals
1. I want to create data that will allow me to
   
   a. analyze the structure of hierarchy across GitHub Repositories (whose opening issues? whose commenting on issues? how many are there overall?)
   
   b. track the sequence of participation for each issue
   
   c. link PRs to issues
   
   d. collect covariates related to issues so I can measure "issue difficulty"
   

In [1]:
from IPython.display import display, HTML

def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n","<br>") ) )

In [2]:
## Import Libraries and Data

In [3]:
import glob
import dask.dataframe as dd
import pandas as pd
from pandarallel import pandarallel
import matplotlib.pyplot as plt
from datetime import datetime, timezone
import ast
import numpy as np
from operator import itemgetter
from stargazer.stargazer import Stargazer

In [4]:
plt.style.use("seaborn")

In [5]:
pandarallel.initialize(progress_bar = True)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
pd.set_option('display.max_columns', None)

In [7]:
%%time
# Read data on issue comments, issues
issue_com = glob.glob('data/github_clean/filtered_github_data/issueCo*')
issue_com.extend(glob.glob('data/github_clean/github_data_pre_18/issueCo*'))
df_issue_comments = pd.concat([pd.read_csv(ele, index_col = 0) for ele in issue_com]).reset_index(drop = True)

issues = glob.glob('data/github_clean/filtered_github_data/issues*')
issues.extend(glob.glob('data/github_clean/github_data_pre_18/issues*'))
df_issue = pd.concat([pd.read_csv(ele, index_col = 0) for ele in issues]).reset_index(drop = True)

CPU times: user 3min 9s, sys: 29 s, total: 3min 38s
Wall time: 4min 21s


# Reduce Memory

In [8]:
# Reduce memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':  # for integers
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:  # for floats.
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## Storing Data

In [9]:
OriginalDataStatistics = pd.DataFrame()
DataDescriptives = pd.DataFrame()

## Data Cleaning

In [10]:
# columns to rename
mod_columns = [ele for ele in df_issue_comments.columns if 'latest' in ele]
mod_dict = {ele : ele.replace('latest_', '') for ele in mod_columns}
df_issue_comments.rename(mod_dict, axis = 1, inplace = True)

In [11]:
full_issue_data = pd.concat([df_issue,df_issue_comments])

In [12]:
# clean data: remove entries with NA issue number values  
df_issue_clean = full_issue_data[~full_issue_data['issue_number'].isna()]
# clean data: add key variable
df_issue_clean['key'] = df_issue_clean['repo_id'].apply(str) + "_" + df_issue_clean['issue_number'].apply(str)

/tmp/user/20506/ipykernel_2655443/1104531115.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_issue_clean['key'] = df_issue_clean['repo_id'].apply(str) + "_" + df_issue_clean['issue_number'].apply(str)


In [13]:
df_issue_clean['created_at'] = pd.to_datetime(df_issue_clean['created_at'])
df_issue_clean['created_at'] = df_issue_clean['created_at'].dt.tz_localize(None)

/tmp/user/20506/ipykernel_2655443/3002124751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_issue_clean['created_at'] = pd.to_datetime(df_issue_clean['created_at'])
/tmp/user/20506/ipykernel_2655443/3002124751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_issue_clean['created_at'] = df_issue_clean['created_at'].dt.tz_localize(None)


In [14]:
# FIX people who aren't classified as owner
fix_ind = df_issue_clean[df_issue_clean.apply(lambda x: x['repo_name'].split("/")[0] == x['issue_user_login'], axis = 1)].index
df_issue_clean.loc[fix_ind, 'issue_author_association'] = 'OWNER'

fix_ind_actor = df_issue_clean[df_issue_clean.apply(lambda x: x['repo_name'].split("/")[0] == x['actor_login'], axis = 1)].index
df_issue_clean.loc[fix_ind_actor, 'actor_repo_association'] = 'OWNER'

In [15]:
df_actor_missing = df_issue_clean[['actor_repo_association', 'actor_login', 'key']].drop_duplicates()
df_actor_missing['count'] = df_actor_missing.groupby(['actor_login','key']).transform('count')

In [16]:
df_actor_missing

,actor_repo_association,actor_login,key,count
0,OWNER,tkalus,96523010_197.0,2
1,NaN,pyup-bot,54683816_1.0,0
2,NaN,Murukarthick,97612481_440.0,1
3,OWNER,DeniRibicic,97612481_736.0,1
4,NaN,lorena1976,33614304_1192.0,0
...,...,...,...,...
3966532,COLLABORATOR,cycleuser,62860862_5.0,1
3966534,NONE,coveralls,101275731_20.0,1
3966537,COLLABORATOR,markotoplak,53335377_182.0,1
3966539,NaN,dwt,1372698_2.0,0


In [17]:
# issue_author_association - author association for individual who opened the issue
# actor_repo_association - author association for individual associated with repository
# need to add actor_repo_association for IssuesEvent
# for "opened" action, just fill in with issue_author_association
opened_issues = df_issue_clean[(df_issue_clean['type'] == 'IssuesEvent') & (df_issue_clean['issue_action'] == 'opened')].index
df_issue_clean.loc[opened_issues, 'actor_repo_association'] = df_issue_clean.loc[opened_issues, 'issue_author_association']
# for "closed" action, need to impute - create table of ranks throughout time

In [18]:
all_ranks = df_issue_clean[((df_issue_clean['type'] == 'IssuesEvent') & (df_issue_clean['issue_action'] == 'opened')) | 
    (df_issue_clean['type'] == 'IssueCommentEvent')][[
    'actor_login', 'created_at', 'actor_repo_association', 'repo_name']].drop_duplicates().sort_values(
    ['actor_login', 'repo_name', 'created_at']).dropna().reset_index(drop = True)

In [19]:
all_ranks = all_ranks.sort_values(['actor_login', 'repo_name', 'created_at'])

In [20]:
all_ranks_change = all_ranks.groupby(['actor_login', 'repo_name', 'actor_repo_association'])['created_at'].min().reset_index()

In [21]:
all_ranks_change['min_date'] = all_ranks_change.groupby(['actor_login', 'repo_name'])['created_at'].transform('min')
all_ranks_change['max_date'] = all_ranks_change.groupby(['actor_login', 'repo_name'])['created_at'].transform('max')

In [22]:
all_ranks_change['count'] = all_ranks_change.sort_values('created_at').groupby(['actor_login', 'repo_name']).cumcount()+1
all_ranks_change['max_count'] = all_ranks_change.sort_values('created_at').groupby(['actor_login', 'repo_name']).transform('count')['count']

In [23]:
# remove people who suddenly become NONE after being ranked
all_ranks_change = all_ranks_change[~all_ranks_change.apply(lambda x: x['actor_repo_association'] == 'NONE' and x['count']>1 and x['count']<=x['max_count'],
                                       axis = 1)]

In [24]:
all_ranks_change = all_ranks_change.sort_values(
    by = ['actor_login', 'repo_name','created_at'])

In [25]:
all_ranks_change['interval_end'] = all_ranks_change.sort_values(
    by = ['created_at']).groupby(
    ['actor_login', 'repo_name'], sort = True)['created_at'].shift(-1)

In [26]:
all_ranks_change.loc[all_ranks_change[all_ranks_change['interval_end'].isna()].index,'interval_end'] = datetime(2023, 8, 31, 11, 59, 59)

In [27]:
## some people are owners when they should not be...
# remove if they're an owner when they're not supposed to be... 
all_ranks_change = all_ranks_change[(all_ranks_change['actor_repo_association'] != 'OWNER') |
    (all_ranks_change.apply(lambda x: x['actor_login'] == x['repo_name'].split("/")[0], axis = 1))]

In [28]:
all_ranks_change['interval_actor'] = all_ranks_change.apply(
    lambda x: [x['actor_repo_association'], x['created_at'], x['interval_end']], axis = 1)
grouped_rank = all_ranks_change.sort_values(['actor_login', 'repo_name', 'created_at']).groupby(
    ['actor_login', 'repo_name', 'min_date', 'max_date']).agg({'interval_actor':list}).reset_index().set_index(['actor_login', 'repo_name'])
grouped_rank['min_date'] = pd.to_datetime(grouped_rank['min_date'])
grouped_rank['max_date'] = pd.to_datetime(grouped_rank['max_date'])

In [29]:
grouped_rank = grouped_rank.reset_index()

In [30]:
grouped_rank['rank_max_date_actor'] = grouped_rank.apply(lambda x: [x['max_date'], x['interval_actor'][-1][0]], axis = 1)
grouped_rank['rank_min_date_actor'] = grouped_rank.apply(
    lambda x: [x['max_date'], 'NONE'] if x['actor_login'] != x['repo_name'].split("/")[0] else [x['max_date'], 'OWNER'], axis = 1)
grouped_rank['rank_max_date_issue'] = grouped_rank['rank_max_date_actor']
grouped_rank['rank_min_date_issue'] = grouped_rank['rank_min_date_actor']
grouped_rank['interval_issue'] = grouped_rank['interval_actor']

In [31]:
df_issue_clean = pd.merge(df_issue_clean, grouped_rank[['actor_login', 'repo_name', 'rank_max_date_actor', 'rank_min_date_actor', 'interval_actor']],
                          how = 'left', on = ['actor_login', 'repo_name'])

df_issue_clean = pd.merge(df_issue_clean, grouped_rank[['actor_login', 'repo_name', 'rank_max_date_issue', 'rank_min_date_issue', 'interval_issue']].rename({
    'actor_login':'issue_user_login'}, axis = 1),
                          how = 'left', on = ['issue_user_login', 'repo_name'])

In [32]:
"""df_issue_clean = df_issue_clean.drop(['rank_max_date_x','rank_min_date_x','interval_x'], axis = 1).rename({
    'rank_max_date_y':'rank_max_date',
    'rank_min_date_y':'rank_min_date',
    'interval_y':'interval'}, axis = 1)"""

"df_issue_clean = df_issue_clean.drop(['rank_max_date_x','rank_min_date_x','interval_x'], axis = 1).rename({\n    'rank_max_date_y':'rank_max_date',\n    'rank_min_date_y':'rank_min_date',\n    'interval_y':'interval'}, axis = 1)"

In [33]:
df_issue_clean['actor_repo_association_repaired'] = df_issue_clean['actor_repo_association']
na_repo_association = df_issue_clean[['rank_max_date_actor', 'rank_min_date_actor', 'interval_actor']].dropna().index
na_issue_repo_association = df_issue_clean[['rank_max_date_issue', 'rank_min_date_issue', 'interval_issue']].dropna().index

In [34]:
print(df_issue_clean['actor_repo_association_repaired'].isna().mean(),
      df_issue_clean['actor_repo_association_repaired'].isna().sum())

0.24620620437992236 1246916


In [35]:
print(df_issue_clean.loc[na_repo_association,'actor_repo_association_repaired'].isna().mean(),
      df_issue_clean.loc[na_repo_association,'actor_repo_association_repaired'].isna().sum())

0.17998320522546507 829895


In [36]:
for col in ['rank_max_date_actor', 'rank_min_date_actor']:
    df_issue_clean.loc[na_repo_association, col] = df_issue_clean.loc[na_repo_association, col].apply(
        lambda x: [pd.to_datetime(x[0]), x[1]])
for col in ['rank_max_date_issue', 'rank_min_date_issue']:
    df_issue_clean.loc[na_issue_repo_association, col] = df_issue_clean.loc[na_issue_repo_association, col].apply(
        lambda x: [pd.to_datetime(x[0]), x[1]])

In [37]:
df_issue_clean.loc[na_repo_association, 'actor_repo_association_repaired'] = df_issue_clean.loc[na_repo_association].apply(
    lambda x: x['rank_min_date_actor'][1] if (x['created_at']<x['rank_min_date_actor'][0]) else 
    x['rank_max_date_actor'][1] if x['created_at']>=x['rank_max_date_actor'][0] else
    [ele[0] for ele in x['interval_actor'] if x['created_at'] >= ele[1] and x['created_at'] < ele[2]], axis = 1)
df_issue_clean.loc[na_repo_association, 'actor_repo_association_repaired'] = df_issue_clean.loc[na_repo_association, 
    'actor_repo_association_repaired'].apply(lambda x: x[0] if type(x) == list and len(x)>=1 else np.nan if type(x) == list else x)

In [38]:
#df_issue_clean.to_csv('df_issue_clean.csv')

In [39]:
#df_issue_clean = pd.read_csv('df_issue_clean.csv', index_col = 0)

In [40]:
df_issue_clean['issue_author_association_repaired'] = df_issue_clean['issue_author_association'] 

In [41]:
df_issue_clean.loc[na_issue_repo_association, 'issue_author_association_repaired'] = \
    df_issue_clean.loc[na_issue_repo_association].apply(
    lambda x: x['rank_min_date_issue'][1] if (x['created_at']<x['rank_min_date_issue'][0]) else 
    x['rank_max_date_issue'][1] if x['created_at']>=x['rank_max_date_issue'][0] else
    [ele[0] for ele in x['interval_issue'] if x['created_at'] >= ele[1] and x['created_at'] < ele[2]], axis = 1)
df_issue_clean.loc[na_issue_repo_association, 'issue_author_association_repaired'] = df_issue_clean.loc[na_issue_repo_association,
    'issue_author_association_repaired'].apply(lambda x: x[0] if type(x) == list and len(x)>=1 else np.nan if type(x) == list else x)



In [42]:
print(df_issue_clean['actor_repo_association_repaired'].isna().mean(),
      df_issue_clean['actor_repo_association_repaired'].isna().sum())
print(df_issue_clean['issue_author_association_repaired'].isna().mean(),
      df_issue_clean['issue_author_association_repaired'].isna().sum())

0.08234167943688235 417021
0.12564885233918563 636351


In [43]:
print(df_issue_clean.loc[na_repo_association,'actor_repo_association_repaired'].isna().mean(),
      df_issue_clean.loc[na_repo_association,'actor_repo_association_repaired'].isna().sum())
print(df_issue_clean.loc[na_issue_repo_association,'issue_author_association_repaired'].isna().mean(),
      df_issue_clean.loc[na_issue_repo_association,'issue_author_association_repaired'].isna().sum())

0.0 0
0.0 0


In [44]:
import warnings
warnings.filterwarnings("ignore")

In [45]:
df_export = df_issue_clean[['created_at', 'type', 'repo_id', 'repo_name', 'actor_login', 'actor_id', 'org_id', 'org_login',
                'issue_user_login', 'issue_user_id','issue_action','issue_assignee','issue_assignees',
                'issue_comment_count','issue_closed_at','issue_number','issue_state', 
                'issue_pull_request','actor_repo_association_repaired','issue_author_association_repaired']]
df_export['issue_comment_count'] = pd.to_numeric(df_export['issue_comment_count'],errors = 'coerce')
#df_export = reduce_mem_usage(df_export)

In [46]:
df_export.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5064519 entries, 0 to 5064518
Data columns (total 20 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   created_at                         datetime64[ns]
 1   type                               object        
 2   repo_id                            int64         
 3   repo_name                          object        
 4   actor_login                        object        
 5   actor_id                           int64         
 6   org_id                             float64       
 7   org_login                          object        
 8   issue_user_login                   object        
 9   issue_user_id                      float64       
 10  issue_action                       object        
 11  issue_assignee                     object        
 12  issue_assignees                    object        
 13  issue_comment_count                float64       
 14  is

In [47]:
df_export.to_parquet('data/merged_data/issue_data.parquet', engine = 'pyarrow')

In [48]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
# Some Statistics on Issues
uq_issue_comments = df_issue_clean[df_issue_clean['type'] == 'IssueCommentEvent']['key'].unique().tolist()
uq_issues = df_issue_clean[df_issue_clean['type'] == 'IssuesEvent']['key'].unique().tolist()
OriginalDataStatistics.loc['Unique Issues (from comments data)', 'count'] = len(uq_issue_comments)
OriginalDataStatistics.loc['Unique Issues (from issues data)', 'count'] = len(uq_issues)
uq_issue_comments.extend(uq_issues)
uq_issue_comments = list(set(uq_issue_comments))
OriginalDataStatistics.loc['Unique Issues (from all data)']  = len(uq_issue_comments)

In [ ]:
OriginalDataStatistics

In [ ]:
# total number of issues 
DataDescriptives.loc[0, 'Total # of Issues'] = df_issue_clean['key'].unique().shape[0]
nunique_issues = df_issue_clean.groupby('repo_name').agg({'issue_number':'nunique'})
DataDescriptives.loc[0, 'Total # of Projects'] = nunique_issues.shape[0]
DataDescriptives.loc[0, 'Average # of Issues/Project'] = nunique_issues.mean()[0]
DataDescriptives.loc[0,"Median # of Issues/Project"] = nunique_issues.median()[0]
DataDescriptives.style.set_caption("Data Descriptives")
DataDescriptives.round(2).to_markdown('descriptives/issues/aggregate_statistics.md')
DataDescriptives

In [ ]:
# PLOT ISSUE COUNT PER REPOSITORY
# Initialize layout
fig, ax = plt.subplots()
#plot
ax.hist(nunique_issues['issue_number'], bins=20, edgecolor="black")
ax.set_title('Number of issues/repository')
ax.set_xlabel('Number of Issues')
ax.set_ylabel('Number of Repositories')
ax.set_yscale('log')
plt.savefig('descriptives/issues/issue_repository.png')
plt.show()

In [ ]:
# PROPORTION OF ALL ISSUES CONTAINED IN top 300 of repos with most issues
(nunique_issues/nunique_issues.sum()).sort_values('issue_number', ascending = False).head(300).sum()

In [ ]:
# LATEST STATUS FOR ISSUE
df_issue_clean['created_at'] = pd.to_datetime(df_issue_clean['created_at'])
df_issue_clean = df_issue_clean.sort_values('created_at', ascending = False)
df_issue_latest_status = df_issue_clean[~df_issue_clean.duplicated(['repo_id', 'issue_number'])]
print(df_issue_latest_status['issue_action'].rename('Issue Latest Status').value_counts(normalize = True).round(3).to_markdown())

In [ ]:
closed_issues = df_issue_latest_status[df_issue_latest_status['issue_action'] == 'closed']['key'].tolist()
# includes open, reopened
open_issues = df_issue_latest_status[df_issue_latest_status['issue_action'] != 'closed']['key'].tolist()

status_dates = df_issue_clean.groupby(['repo_name', 'issue_number', 'key', 'issue_action']).agg({'created_at': ['min', 'max']}).reset_index()
status_dates.columns = ['repo_name', 'issue_number', 'key', 'issue_action', 'min_date', 'max_date']
status_dates = status_dates.pivot(index = ['repo_name', 'issue_number', 'key'], columns = 'issue_action', values = ['min_date' ,'max_date']).reset_index()
status_dates.columns = ['repo_name', 'issue_number', 'key', 'closed_min_date', 'opened_min_date', 'reopened_min_date', 
                        'closed_max_date', 'opened_max_date', 'reopened_max_date']
# remove ones that do not have an opened_min_date
status_dates = status_dates[~status_dates['opened_min_date'].isna()]
# ones that are closed - time open: use earliest open date minus latest close date
closed_ind = status_dates[status_dates['key'].isin(closed_issues)].index
status_dates.loc[closed_ind, 'open_time'] = status_dates.loc[closed_ind, 'closed_max_date'] - \
    status_dates.loc[closed_ind, 'opened_min_date']
# ones that are still open - time open: use 8/31/2023 - opened date
open_ind = status_dates[status_dates['key'].isin(open_issues)].index
last_date = datetime(2023, 8, 31, 11, 59, 59)
status_dates.loc[open_ind, 'open_time'] = last_date - status_dates.loc[open_ind, 'opened_min_date']

# type of issue
status_dates.loc[closed_ind, 'status'] = 'closed'
status_dates.loc[open_ind, 'status'] = 'open'
status_dates['open_time_days'] = status_dates['open_time'].apply(lambda x: x.days)

### PLOT EXCLUDES ISSUES THAT WE DO NOT HAVE OPENED/CLOSED dates for
# Initialize layout
fig, ax = plt.subplots(1, 2, figsize = (10, 4))
#plot
status_dates.hist('open_time_days', by = 'status', bins=20, edgecolor="black", ax = ax)
fig.suptitle('Histogram of Days Open for Issues')
for a in ax:
    a.set_ylabel('Number of Issues')
    a.set_xlabel('Days Open')
    #a.set_yscale('log')
ax[0].set_title('Closed Issues (as of 8/31/2023)')
ax[1].set_title('Open Issues (as of 8/31/2023)')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
def lessThanGroups(status_dates):
    og_shape = status_dates.shape[0]
    one_day = status_dates[status_dates['open_time_days']<=1].shape[0]/og_shape
    one_week = status_dates[status_dates['open_time_days']<=7].shape[0]/og_shape
    one_month = status_dates[status_dates['open_time_days']<=30].shape[0]/og_shape
    six_month = status_dates[status_dates['open_time_days']<=180].shape[0]/og_shape
    one_year = status_dates[status_dates['open_time_days']<=365].shape[0]/og_shape
    g_one_year = status_dates[status_dates['open_time_days']>365].shape[0]/og_shape
    return [one_day, one_week, one_month, six_month, one_year, g_one_year, og_shape]

In [ ]:
df_open_days = pd.DataFrame([lessThanGroups(status_dates[status_dates['status'] == 'closed']),
              lessThanGroups(status_dates[status_dates['status'] == 'open'])]).round(2)
df_open_days.index = ['closzed issues', 'open issues']
df_open_days.columns = ['open $\leq$ 1 day', 'open $\leq$ 1 week', 'open $\leq$ 1 month', 'open $\leq$ 6 months', \
                        'open $\leq$ 1 year', 'open $>$ 1 year', '# of issues']
df_open_days

In [ ]:
# TEST WHAT HAPPENS WHEN WE "APPROXIMATE" 
closed_issues = df_issue_latest_status[df_issue_latest_status['issue_state'] == 'closed']['key'].tolist()
# includes open, reopened
open_issues = df_issue_latest_status[df_issue_latest_status['issue_state'] != 'closed']['key'].tolist()

status_dates = df_issue_clean.groupby(['repo_name', 'issue_number', 'key', 'issue_state']).agg({'created_at': ['min', 'max']}).reset_index()
status_dates.columns = ['repo_name', 'issue_number', 'key', 'issue_state', 'min_date', 'max_date']
status_dates = status_dates.pivot(index = ['repo_name', 'issue_number', 'key'], columns = 'issue_state', values = ['min_date' ,'max_date']).reset_index()
status_dates.columns = ['repo_name', 'issue_number', 'key', 'closed_min_date', 'opened_min_date', 
                        'closed_max_date', 'opened_max_date']
# remove ones that do not have an opened_min_date
status_dates = status_dates[~status_dates['opened_min_date'].isna()]
# ones that are closed - time open: use earliest open date minus latest close date
closed_ind = status_dates[status_dates['key'].isin(closed_issues)].index
status_dates.loc[closed_ind, 'open_time'] = status_dates.loc[closed_ind, 'closed_max_date'] - \
    status_dates.loc[closed_ind, 'opened_min_date']
# ones that are still open - time open: use 8/31/2023 - opened date
open_ind = status_dates[status_dates['key'].isin(open_issues)].index
last_date = datetime(2023, 8, 31, 11, 59, 59)
status_dates.loc[open_ind, 'open_time'] = last_date - status_dates.loc[open_ind, 'opened_min_date']

# type of issue
status_dates.loc[closed_ind, 'status'] = 'closed'
status_dates.loc[open_ind, 'status'] = 'open'
status_dates['open_time_days'] = status_dates['open_time'].apply(lambda x: x.days)

### PLOT EXCLUDES ISSUES THAT WE DO NOT HAVE OPENED/CLOSED dates for
# Initialize layout
fig, ax = plt.subplots(1, 2, figsize = (10, 4))
#plot
status_dates.hist('open_time_days', by = 'status', bins=20, edgecolor="black", ax = ax)
fig.suptitle('Histogram of Days Open for Issues')
for a in ax:
    a.set_ylabel('Number of Issues')
    a.set_xlabel('Days Open')
    #a.set_yscale('log')
ax[0].set_title('Closed Issues (as of 8/31/2023)')
ax[1].set_title('Open Issues (as of 8/31/2023)')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
df_open_days = pd.DataFrame([lessThanGroups(status_dates[status_dates['status'] == 'closed']),
              lessThanGroups(status_dates[status_dates['status'] == 'open'])]).round(2)
df_open_days.index = ['closed issues', 'open issues']
df_open_days.columns = ['open $\leq$ 1 day', 'open $\leq$ 1 week', 'open $\leq$ 1 month', 'open $\leq$ 6 months', \
                        'open $\leq$ 1 year', 'open $>$ 1 year', '# of issues']
df_open_days

## Whose Opening Issues

In [ ]:
opened_issues = df_issue_clean.sort_values(['key', 'created_at'])[['key', 'issue_author_association_repaired']].dropna().drop_duplicates()


In [ ]:
def groupActions(df_issue_clean):
    closed_issues = df_issue_clean[df_issue_clean['issue_action'] == 'closed'].sort_values(
    ['key', 'created_at'])[['key', 'issue_author_association_repaired']].drop_duplicates()
    opened_issues = df_issue_clean.sort_values(['key', 'created_at'])[['key', 'issue_author_association_repaired']].dropna().drop_duplicates()
    closed_issues['status'] = 'closed'
    opened_issues['status'] = 'opened'
    all_issues = pd.concat([closed_issues, opened_issues])
    group_actions = all_issues.groupby('status')['issue_author_association_repaired'].value_counts(
        normalize = True).reset_index().pivot(index = 'status', columns = 'issue_author_association_repaired', values = 'proportion')
    group_actions = pd.concat([group_actions, all_issues['status'].value_counts()], axis = 1)
    group_actions.columns.name = ''
    group_actions = group_actions.loc[['closed', 'opened']]
    group_actions.index = ['Closing', 'Opening']
    return group_actions[['NONE', 'CONTRIBUTOR', 'MEMBER', 'COLLABORATOR', 'OWNER', 'count']], all_issues

In [ ]:
tb, all_issues = groupActions(df_issue_clean)
tb.round(2)

In [ ]:
all_issues['repo'] = all_issues['key'].parallel_apply(lambda x: x.split("_")[0])
# percent opened by owner vs. # of issues
# percent opened by NONE vs. # of issues
all_issues['opened_owner'] = all_issues['issue_author_association_repaired'] == 'OWNER'
all_issues['opened_none'] = all_issues['issue_author_association_repaired'] == 'NONE'

In [ ]:
grouped_all_issues = all_issues.groupby('repo').agg({'opened_owner': ['count', 'mean'],'opened_none':['mean']})
grouped_all_issues.columns = ['issue_count', 'opened_owner_mean', 'opened_none_mean']

In [ ]:
### PLOT EXCLUDES ISSUES THAT WE DO NOT HAVE OPENED/CLOSED dates for
# Initialize layout
fig, ax = plt.subplots(2, 2, figsize = (10, 8))
#plot
ax[0,0].scatter(x = grouped_all_issues['issue_count'], y = grouped_all_issues['opened_owner_mean'])
ax[0,1].scatter(x = grouped_all_issues['issue_count'], y = grouped_all_issues['opened_none_mean'])
ax[1,0].scatter(x = grouped_all_issues.query('issue_count>1000')['issue_count'], 
                y = grouped_all_issues.query('issue_count>1000')['opened_owner_mean'])
ax[1,1].scatter(x = grouped_all_issues.query('issue_count>1000')['issue_count'], 
                y = grouped_all_issues.query('issue_count>1000')['opened_none_mean'])

for i in range(2):
    ax[i, 0].set_ylabel('% of Issues Opened by Owner')
    ax[i, 1].set_ylabel('% of Issues Opened by Unaffiliated Contributors')

    ax[i, 0].set_xlabel('Number of Issues')
    ax[i, 1].set_xlabel('Number of Issues')
    ax[1, i].set_title('Min 1000 Issues')
fig.suptitle("% of Issues Opened by Group vs. Issue Count")
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
tb_ray, all_issues_ray = groupActions(df_issue_clean[df_issue_clean['repo_name'] == 'ray-project/ray'])
tb_ray.round(2)

# Existence of Hierarchy

Activity is defined as opening, closing and commenting on an issue

In [ ]:
explicit_open_keys = df_issue_clean.query('issue_action == "opened"')['key'].unique().tolist()
remaining_issues = df_issue_clean[~df_issue_clean['key'].isin(explicit_open_keys)][['key', 'issue_author_association']].drop_duplicates()['issue_author_association']

In [ ]:
def make_generated_activity_base(df_issue_clean):
    rank_identifying_cols = ['actor_login', 'repo_name', 'actor_repo_association_repaired']
    generated_activity_base = pd.concat([
        df_issue_clean[['key', 'issue_author_association_repaired']].drop_duplicates()['issue_author_association_repaired'].value_counts(),
        df_issue_clean.query('issue_action == "closed"')['actor_repo_association_repaired'].value_counts(),
        df_issue_clean.query('issue_action == "closed" and actor_login == issue_user_login')['actor_repo_association_repaired'].value_counts(),
        df_issue_clean.query('issue_action == "closed" and actor_login != issue_user_login')['actor_repo_association_repaired'].value_counts(),
        df_issue_clean.query('type == "IssueCommentEvent"')['actor_repo_association_repaired'].value_counts(),
        df_issue_clean.query('type == "IssueCommentEvent" and actor_login == issue_user_login')['actor_repo_association_repaired'].value_counts(),
        df_issue_clean.query('type == "IssueCommentEvent" and actor_login != issue_user_login')['actor_repo_association_repaired'].value_counts(),
        df_issue_clean[rank_identifying_cols].drop_duplicates()['actor_repo_association_repaired'].value_counts(),
    ], axis = 1).round(2).loc[
        ['NONE', 'CONTRIBUTOR', 'MEMBER', 'COLLABORATOR', 'OWNER']]
    generated_activity_base.columns = ['# of Opened Issues', '# of Closed Issues', '# of Closed Issues (Own Issue)', '# of Closed Issues (Others Issue)', 
                                       '# of Comments', '# of Comments (Own Issue)', '# of Comments (Others Issue)', '# of Individiuals']
    generated_activity_base['Amount of Activity'] = generated_activity_base.apply(
        lambda x: x[['# of Opened Issues', '# of Closed Issues', '# of Comments']].sum(), axis = 1)
    generated_activity_base['Proportion'] = generated_activity_base['Amount of Activity']/generated_activity_base['Amount of Activity'].sum()
    generated_activity_base = generated_activity_base[['Amount of Activity', 'Proportion', '# of Opened Issues', '# of Closed Issues', 
                                                       '# of Closed Issues (Own Issue)', '# of Closed Issues (Others Issue)', 
                                                       '# of Comments', '# of Comments (Own Issue)', '# of Comments (Others Issue)', '# of Individiuals']]
    return generated_activity_base

In [ ]:
generated_activity_base = make_generated_activity_base(df_issue_clean)

In [ ]:
generated_activity = generated_activity_base.copy()
generated_activity['Activity/Individual'] = generated_activity['Amount of Activity']/generated_activity['# of Individiuals']
for col in ['Opened Issues', 'Closed Issues', 'Closed Issues (Own Issue)',  'Closed Issues (Others Issue)',
            'Comments', 'Comments (Own Issue)', 'Comments (Others Issue)']:
    generated_activity[f'{col}/ Individual'] = generated_activity.apply(
        lambda x: f"{x[f'# of {col}']/x['# of Individiuals']:.2f}\n({100*x[f'# of {col}']/x['Amount of Activity']:.2f}%)", axis = 1)
for col in ['# of Opened Issues', '# of Closed Issues', '# of Closed Issues (Own Issue)', 
            '# of Closed Issues (Others Issue)', '# of Comments', '# of Comments (Own Issue)', 
            '# of Comments (Others Issue)', '# of Individiuals']:
    generated_activity[col] = generated_activity[col].apply(lambda x: f"{x}\n({x/generated_activity[col].sum():.2f})")

In [ ]:
pretty_print(generated_activity.round(2))

Now, I'm going to weight the individual count by how long they were that rank for. I do this because I know in some weird instances, we see people obtain ranks for very short periods of time. This will allow me to more precisely understand how much activity an individual contributed to. 

In [ ]:
all_users = pd.concat([df_issue_clean[['created_at', 'actor_login', 'repo_name']].rename({'actor_login':'user'}, axis = 1),
                       df_issue_clean[['created_at', 'issue_user_login', 'repo_name']].rename({'issue_user_login':'user'}, axis = 1)])
earliest = all_users.sort_values('created_at').drop_duplicates(['user', 'repo_name']).rename({'created_at': 'earliest_date'}, axis = 1)
latest = all_users.sort_values('created_at', ascending = False).drop_duplicates(['user', 'repo_name']).rename({'created_at': 'latest_date'}, axis = 1)
user_dates = pd.merge(latest, earliest, on = ['user', 'repo_name'], how = 'outer')[['user', 'repo_name', 'earliest_date', 'latest_date']]

In [ ]:
user_dates['key'] = user_dates['user']+"_"+user_dates['repo_name']
early_dict = user_dates.set_index('key')['earliest_date'].to_dict()
last_dict = user_dates.set_index('key')['latest_date'].to_dict()

In [ ]:
from pandas import Timestamp
for col in grouped_rank.columns[4:10]:
    print(col)
    grouped_rank[col] = grouped_rank[col].parallel_apply(lambda x: eval(x, {'datetime': datetime,
                                                                            'Timestamp': Timestamp}))

In [ ]:
def make_time_spent(grouped_rank):
    time_spent = pd.DataFrame(grouped_rank['final_interval'].parallel_apply(
        lambda x: [[ele[0], pd.to_datetime(ele[2])-pd.to_datetime(ele[1])] for ele in x]).explode())
    time_spent['rank'] = time_spent['final_interval'].apply(lambda x: x[0])
    time_spent['time spent'] = time_spent['final_interval'].apply(lambda x: x[1])
    time_spent['Time Spent Seconds'] = time_spent['time spent'].apply(lambda x: x.total_seconds())
    time_spent['Time Spent (Prop.)'] = time_spent['Time Spent Seconds']/time_spent['Time Spent Seconds'].sum()
    time_spent['Time Spent (Worker-Years)'] = time_spent['Time Spent Seconds']/(86400*30*12)

    return time_spent

In [ ]:
grouped_rank['key'] = grouped_rank['actor_login']+"_"+grouped_rank['repo_name']
grouped_rank['final_interval'] = grouped_rank.parallel_apply(
    lambda x: [[x['interval_actor'][0][0], early_dict[x['key']], last_dict[x['key']]]] if len(x['interval_actor']) == 1 else 
    [[x['interval_actor'][0][0], early_dict[x['key']], x['interval_actor'][0][2]]] + x['interval_actor'][1:-1] + \
    [[x['interval_actor'][-1][0], x['interval_actor'][-1][1], last_dict[x['key']]]], axis = 1)

time_spent = make_time_spent(grouped_rank)

In [ ]:
def make_generated_activity_time(generated_activity_base, time_spent):
    generated_activity_time = pd.concat([generated_activity_base, time_spent.groupby('rank')['Time Spent (Worker-Years)'].sum()], axis = 1)
    generated_activity_time['Activity/Worker-Year'] = generated_activity_time['Amount of Activity']/generated_activity_time['Time Spent (Worker-Years)']
    for col in ['Opened Issues', 'Closed Issues', 'Closed Issues (Own Issue)',  'Closed Issues (Others Issue)',
                'Comments', 'Comments (Own Issue)', 'Comments (Others Issue)']:
        generated_activity_time[f'{col}/Time Spent'] = generated_activity_time.apply(
            lambda x: f"{x[f'# of {col}']/x['Time Spent (Worker-Years)']:.2f}\n ({100*x[f'# of {col}']/x['Amount of Activity']:.2f}%)", axis = 1)
    
    
    for col in ['# of Opened Issues', '# of Closed Issues', '# of Closed Issues (Own Issue)', 
                '# of Closed Issues (Others Issue)', '# of Comments', '# of Comments (Own Issue)', 
                '# of Comments (Others Issue)', '# of Individiuals']:
        generated_activity_time[col] = generated_activity_time[col].apply(lambda x: f"{x:,}\n({x/generated_activity_time[col].sum():.2f})")

    generated_activity_time['Amount of Activity'] = generated_activity_time['Amount of Activity'].apply(lambda x: f"{x:,}")
    generated_activity_time['Proportion'] = generated_activity_time['Proportion'].round(2)

    generated_activity_time['Time Spent (Worker-Years)'] = generated_activity_time['Time Spent (Worker-Years)'].apply(
        lambda x: f"{x:,.2f}\n({x/generated_activity_time['Time Spent (Worker-Years)'].sum():.2f})")
    generated_activity_time['Activity/Worker-Year'] = generated_activity_time['Activity/Worker-Year'].round(2)

    return generated_activity_time

In [ ]:
generated_activity_time = make_generated_activity_time(generated_activity_base, time_spent)


main difference between this and a previous analysis is that owners become more involved if you consider the "intensity" of 
someone's actions

ie: how much work someone does within a given time period

I prefer this one because it makes individuals more comparable within a particular time period

**most important findings:**
1) Large Disparity in Activity/Individual - Higher ranked individuals do more things in the same relative time period
2) Individuals progressively spend 
    a) less of their time opening issues (asking questions/flagging problems)
    b) more of their time closing issues (administrative work). In particular its notable how many issues owners close, 
    especially given that they do less things. Owners close A LOT of issues..
3) As individuals rise in rank,
    a) they spend more time commenting on issues, especially other people's issues
    b) they don't spend that much time commenting on their own issues - why is that?

In [ ]:
pretty_print(generated_activity_time.round(2))

In [ ]:
pretty_print(generated_activity_time[['Amount of Activity','Proportion','# of Opened Issues','# of Closed Issues',
                         '# of Closed Issues (Own Issue)','# of Closed Issues (Others Issue)','# of Comments','# of Comments (Own Issue)',
                         '# of Comments (Others Issue)']])

In [ ]:
pretty_print(generated_activity_time[['Amount of Activity','Time Spent (Worker-Years)','Activity/Worker-Year','Opened Issues/Time Spent',
                                      'Closed Issues/Time Spent','Closed Issues (Own Issue)/Time Spent','Closed Issues (Others Issue)/Time Spent','Comments/Time Spent',
                                      'Comments (Own Issue)/Time Spent','Comments (Others Issue)/Time Spent']])

In [ ]:
# how does it vary by repository size?

In [ ]:
repo_contributors = df_issue_clean[['repo_name', 'actor_login']].drop_duplicates()['repo_name'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize = (4, 3))
pd.Series(repo_contributors.values).hist(ax = ax, bins = 30)
ax.set_title('Histogram of # of Contributors to Repositories')
ax.set_xlabel('Number of Unique Contributors')
ax.set_ylabel('Number of Repositories')
ax.set_yscale('log')
plt.show()

## Time for More Analysis

The main difference between this and a previous analysis is that owners become more involved if you consider the "intensity" of 
someone's actions (how much work someone does within a given time period). I prefer this one because it makes individuals more comparable within a particular time period

**most important findings:**
1) Large Disparity in Activity/Individual - Higher ranked individuals do more things in the same relative time period
2) Individuals progressively spend
   
    a) less of their time opening issues (asking questions/flagging problems)
   
    b) more of their time closing issues (administrative work). In particular its notable how many issues owners close, 
    especially given that they do less things. Owners close A LOT of issues..
4) As individuals rise in rank,

    a) they spend more time commenting on issues, especially other people's issues

    b) they don't spend that much time commenting on their own issues - why is that?

1) Why do they do more things?

2) 

In [ ]:
pretty_print(generated_activity_time.round(2))

In [ ]:
organization_list = df_issue_clean['org_login'].dropna().unique()

In [ ]:
watch_lst = glob.glob('data/github_clean/github_data_pre_18/watchEvent*')
watch_lst.extend(glob.glob('data/github_clean/filtered_github_data/watchEvent*'))
watch_data = pd.concat([pd.read_csv(file, usecols = ['repo_name'], engine = 'pyarrow') for file in watch_lst])
watch_data = watch_data.value_counts()

In [ ]:
fork_lst = glob.glob('data/github_clean/github_data_pre_18/forkEvent*')
fork_lst.extend(glob.glob('data/github_clean/filtered_github_data/forkEvent*'))
fork_data = pd.concat([pd.read_csv(file, usecols = ['repo_name'], engine = 'pyarrow') for file in fork_lst])
fork_data = fork_data.value_counts()

In [ ]:
def calculateTimeSpent(grouped_rank):
    time_spent = pd.DataFrame(grouped_rank['final_interval'].parallel_apply(
        lambda x: [[ele[0], pd.to_datetime(ele[2])-pd.to_datetime(ele[1])] for ele in x]).explode())
    time_spent['rank'] = time_spent['final_interval'].apply(lambda x: x[0])
    time_spent['time spent'] = time_spent['final_interval'].apply(lambda x: x[1])
    time_spent['Time Spent Seconds'] = time_spent['time spent'].apply(lambda x: x.total_seconds())
    time_spent['Time Spent (Prop.)'] = time_spent['Time Spent Seconds']/time_spent['Time Spent Seconds'].sum()
    time_spent['Time Spent (Worker-Years)'] = time_spent['Time Spent Seconds']/(86400*30*12)
    return time_spent

In [ ]:
time_spent_org = calculateTimeSpent(grouped_rank[grouped_rank['repo_name'].apply(lambda x: x.split("/")[0] in organization_list)])
time_spent_noorg = calculateTimeSpent(grouped_rank[~grouped_rank['repo_name'].apply(lambda x: x.split("/")[0] in organization_list)])
time_spent_many_contributor = calculateTimeSpent(grouped_rank[grouped_rank['repo_name'].isin(repo_contributors.head(300).index)])
time_spent_no_many = calculateTimeSpent(grouped_rank[~grouped_rank['repo_name'].isin(repo_contributors.head(300).index)])
time_spent_most_activity = calculateTimeSpent(grouped_rank[grouped_rank['repo_name'].isin(
    df_issue_clean['repo_name'].value_counts().head(300).index)])
time_spent_not_most = calculateTimeSpent(grouped_rank[~grouped_rank['repo_name'].isin(
    df_issue_clean['repo_name'].value_counts().head(300).index)])
time_spent_most_watch = calculateTimeSpent(grouped_rank[grouped_rank['repo_name'].isin(
    watch_data.reset_index()['repo_name'].head(300).tolist())])
time_spent_not_watch = calculateTimeSpent(grouped_rank[~grouped_rank['repo_name'].isin(
    watch_data.reset_index()['repo_name'].head(300).tolist())])
time_spent_most_fork = calculateTimeSpent(grouped_rank[grouped_rank['repo_name'].isin(
    fork_data.reset_index()['repo_name'].head(300).tolist())])
time_spent_not_fork = calculateTimeSpent(grouped_rank[~grouped_rank['repo_name'].isin(
    fork_data.reset_index()['repo_name'].head(300).tolist())])

In [ ]:
hierarchy_dist_most = pd.concat([
    time_spent_org.groupby('rank')['Time Spent (Worker-Years)'].sum().rename('Organization'),
    time_spent_many_contributor.groupby('rank')['Time Spent (Worker-Years)'].sum().rename('Top 300 (repos) Most Contributors'),
    time_spent_most_activity.groupby('rank')['Time Spent (Worker-Years)'].sum().rename('Top 300 (repos) Most Activity'),
    time_spent_most_watch.groupby('rank')['Time Spent (Worker-Years)'].sum().rename('Top 300 (repos) Most Watches'),
    time_spent_most_fork.groupby('rank')['Time Spent (Worker-Years)'].sum().rename('Top 300 (repos) Most Forks'),
], axis = 1)
hierarchy_dist_less = pd.concat([
    time_spent_noorg.groupby('rank')['Time Spent (Worker-Years)'].sum().rename('Not Organization'),
    time_spent_no_many.groupby('rank')['Time Spent (Worker-Years)'].sum().rename('Not Top 300 (repos) Most Contributors'),
    time_spent_not_most.groupby('rank')['Time Spent (Worker-Years)'].sum().rename('Not Top 300 (repos) Most Activity'),
    time_spent_not_watch.groupby('rank')['Time Spent (Worker-Years)'].sum().rename('Not Top 300 (repos) Most Watches'),
    time_spent_not_fork.groupby('rank')['Time Spent (Worker-Years)'].sum().rename('Not Top 300 (repos) Most Forks'),
], axis = 1)

for col in hierarchy_dist_most.columns:
    hierarchy_dist_most[col] = hierarchy_dist_most[col].apply(lambda x: f"{x:.0f}\n({x/hierarchy_dist_most[col].sum():.2f})")
for col in hierarchy_dist_less.columns:
    hierarchy_dist_less[col] = hierarchy_dist_less[col].apply(lambda x: f"{x:.0f}\n({x/hierarchy_dist_less[col].sum():.2f})")

hierarchy_dist_most.columns.name = 'Time Spent (Worker-Years)'
hierarchy_dist_less.columns.name = 'Time Spent (Worker-Years)'
pretty_print(hierarchy_dist_most.loc[['NONE', 'CONTRIBUTOR', 'MEMBER','COLLABORATOR', 'OWNER']])

In [ ]:
pretty_print(hierarchy_dist_less.loc[['NONE', 'CONTRIBUTOR', 'MEMBER','COLLABORATOR', 'OWNER']])

In [ ]:
top300_contri_grouped_rank = grouped_rank[grouped_rank['repo_name'].isin(repo_contributors.head(300).index)]
top300_contri_df_issue_clean = df_issue_clean[df_issue_clean['repo_name'].isin(repo_contributors.head(300).index)]
nottop300_contri_grouped_rank = grouped_rank[~grouped_rank['repo_name'].isin(repo_contributors.head(300).index)]
nottop300_contri_df_issue_clean = df_issue_clean[~df_issue_clean['repo_name'].isin(repo_contributors.head(300).index)]

In [ ]:
top300_contri_time_spent = make_time_spent(top300_contri_grouped_rank)
top300_contri_generated_activity_base = make_generated_activity_base(top300_contri_df_issue_clean)
top300_contri_generated_activity_time = make_generated_activity_time(top300_contri_generated_activity_base, top300_contri_time_spent)

In [ ]:
nottop300_contri_time_spent = make_time_spent(nottop300_contri_grouped_rank)
nottop300_contri_generated_activity_base = make_generated_activity_base(nottop300_contri_df_issue_clean)
nottop300_contri_generated_activity_time = make_generated_activity_time(nottop300_contri_generated_activity_base, nottop300_contri_time_spent)

In [ ]:
top300_contri_generated_activity_time.columns.name = 'Top 300 Repositories, by Contributor Count'
pretty_print(top300_contri_generated_activity_time[['Amount of Activity','Proportion','# of Opened Issues','# of Closed Issues',
                         '# of Closed Issues (Own Issue)','# of Closed Issues (Others Issue)','# of Comments','# of Comments (Own Issue)',
                         '# of Comments (Others Issue)']])

In [ ]:
nottop300_contri_generated_activity_time.columns.name = 'All Repositories except Top 300 by Contributor Count'
pretty_print(nottop300_contri_generated_activity_time[['Amount of Activity','Proportion','# of Opened Issues','# of Closed Issues',
                         '# of Closed Issues (Own Issue)','# of Closed Issues (Others Issue)','# of Comments','# of Comments (Own Issue)',
                         '# of Comments (Others Issue)']]) 

In [ ]:
print(top300_contri_df_issue_clean.shape[0]/top300_contri_df_issue_clean['key'].unique().shape[0], top300_contri_df_issue_clean['key'].unique().shape[0])
print(nottop300_contri_df_issue_clean.shape[0]/nottop300_contri_df_issue_clean['key'].unique().shape[0], nottop300_contri_df_issue_clean['key'].unique().shape[0])

In [ ]:
pretty_print(top300_contri_generated_activity_time[['Amount of Activity','Time Spent (Worker-Years)','Activity/Worker-Year','Opened Issues/Time Spent',
                                      'Closed Issues/Time Spent','Closed Issues (Own Issue)/Time Spent','Closed Issues (Others Issue)/Time Spent','Comments/Time Spent',
                                      'Comments (Own Issue)/Time Spent','Comments (Others Issue)/Time Spent']])

In [ ]:
pretty_print(nottop300_contri_generated_activity_time[['Amount of Activity','Time Spent (Worker-Years)','Activity/Worker-Year','Opened Issues/Time Spent',
                                      'Closed Issues/Time Spent','Closed Issues (Own Issue)/Time Spent','Closed Issues (Others Issue)/Time Spent','Comments/Time Spent',
                                      'Comments (Own Issue)/Time Spent','Comments (Others Issue)/Time Spent']])

## Analyzing the Lifetime of an Issue

In [ ]:
non_author_comments = df_issue_clean[(df_issue_clean['type'] == 'IssueCommentEvent') & (df_issue_clean['actor_login'] != df_issue_clean['issue_user_login'])]

In [ ]:
num_contributors = df_issue_clean[['key', 'actor_login']].drop_duplicates()
print(f"{num_contributors.shape[0]/df_issue_clean['key'].unique().shape[0]:.2f} contributors to each issue on average")

In [ ]:
contributors_per_issue = num_contributors.groupby('key')['actor_login'].count()

In [ ]:
fig, ax = plt.subplots()
contributors_per_issue.plot(kind = 'hist', ax = ax, bins = 50)
ax.set_yscale('log')
plt.show()

In [ ]:
first_nonauthor_comment = non_author_comments.sort_values('created_at').drop_duplicates(['key'])

In [ ]:
second_nonauthor_comment = non_author_comments[~non_author_comments.index.isin(first_nonauthor_comment.index)].sort_values(
    'created_at').drop_duplicates(['key'])

In [ ]:
# second commenter normally same as first commenter
# surprisingly a lot of NONEs reply to NONE opened issues (discussion involving unaffiliateds, and then contributors)
# mostly NONEs that respond to OWNER issues (interesting...very unhierarchical)
# otherwise OWNERs rarely respond to issues
# CONTRIBUTOR -> most likely is fellow contributor, then NONE (how to explain?) then MEMBER and then COLLABORATOR
# COLLABORATOR -> most likely is COLLABORATOR, then NONE, then CONTRIBUTOR then MEMBER
# MEMBER IS WEIRD
# similarly MEMBER -> most likely is MEMBER, then NONE, then CONTRIBUTOR, then COLLABORATOR
# i mean general vibe is similar RANK, and then decreasing chance of lower -> higher rank individuals


commentor_author = pd.concat([first_nonauthor_comment.groupby('issue_author_association_repaired')[
                              'actor_repo_association_repaired'].value_counts(),
                              first_nonauthor_comment.groupby('issue_author_association_repaired')[
                              'actor_repo_association_repaired'].value_counts(normalize = True),
                              second_nonauthor_comment.groupby('issue_author_association_repaired')[
                              'actor_repo_association_repaired'].value_counts(),
                              second_nonauthor_comment.groupby('issue_author_association_repaired')[
                              'actor_repo_association_repaired'].value_counts(normalize = True),
          ], axis = 1).loc[[
    'NONE', 'CONTRIBUTOR', 'MEMBER', 'COLLABORATOR', 'OWNER']]
commentor_author = commentor_author.reset_index()
#commentor_author.columns = ['Author Rank', 'Replier Rank', 'First Replier (Count)', 'Second Replier']
commentor_author.round(2)

In [ ]:
# how many different individuals are involved in commenting on an issue? how many commenters are there? How are the two related?
# how often is the closer one of the commenters?
# when do you see the first commenter (second person) arrive? third person? what is their rank
# 

In [ ]:
non_author_comments['earliest_nonauthor_comment_date'] = non_author_comments.groupby(['key', 'created_at'])['created_at'].transform('min')

In [ ]:
non_author_comments[non_author_comments['earliest_nonauthor_comment_date'] == non_author_comments['created_at']].groupby(
    'issue_author_association_repaired')['actor_repo_association_repaired'].value_counts()

In [ ]:
## whose opening issues
## whose commenting
## whose closing issues

df_issue_clean

In [ ]:
break

In [ ]:
# read prs

add smt about PRs (linked PRs = issue that is PR review + PRs I connected)

# now have to dig into PRs....

# who works on the PR (code)
# who works on the PR (commenting on PR review)
# who works on the issue itself (commenting)

# who reviews the PR request

# closed date
# comment info 